In [27]:
import pandas as pd
import numpy as np
# import sqlalchemy
import matplotlib.pyplot as plt
import re
import sys

%matplotlib inline

In [99]:
data = pd.read_csv('MERGED2013_PP.csv')
print "Number of features: {}".format(len(data.columns))
print "Number of rows: {}".format(len(data))

Number of features: 1729
Number of rows: 7804


In [100]:
# potentially interesting features
col_desc = {
    'C150_4_POOLED': 'Completion 4yr pooled',
    'C150_L4_POOLED': 'Completion <4yr pooled',
    'CCSIZSET': 'Carnegie classification-Size & settings',
    'CCUGPROF': 'Carnegie classification-Undergrad profile ',
    'CCBASIC': 'Carnegie classification-basic',
    'LOCALE': 'Degree of urbanization',
    'region': 'Region',
    'PREDDEG': 'Predominant degree awarded',
    'CONTROL': 'Control (public/private)',
    'RET_FT4': 'Retention 4yr',
    'RET_FTL4': 'Retention <4yr',
    'ACTCMMID': 'ACT',
    'SAT_AVG': 'SAT',
    'SAT_AVG_ALL': 'SAT all',
    'SATVRMID': 'SAT reading',
    'SATMTMID': 'SAT math',
    'SATWRMID': 'SAT writing',
    'AVGFACSAL': 'Avg faculty salary',
    'PFTFAC': 'Full time faculty rate',
    'ADM_RATE_ALL': 'Admission rate',
    'DISTANCEONLY': 'Distance only',
    'NPT4_PUB': 'Avg net price title IV institut public',
    'NPT4_PRIV': 'Avg net price title IV institut private',
    'NUM4_PUB': 'Num Title IV student, public',
    'NUM4_PRIV': 'Num Title IV student, private',
    'COSTT4_A': 'Avg cost academic year',
    'COSTT4_P': 'Avg cost program year',
    'TUITIONFEE_IN': 'In state tuition',
    'TUITIONFEE_OUT': 'Out of state tuition',
    'TUITIONFEE_PROG': 'Tuition fee program year',
    'TUITFTE': 'Net revenue per FTE student',
    'INEXPFTE': 'Expense per FTE student',
    'PCTPELL': '% Pell Grant receiver',
    'PCTFLOAN': '% Fed student loan',
    'UG25abv': '% undergrad > 25 yr',
    'PFTFTUG1_EF': 'Undergrad 1st-time degree seeking',
    'UGDS': 'Number of Undergrad degree seeking',
    'PAR_ED_PCT_1STGEN': '% 1st gen students',
    'PAR_ED_PCT_MS': '% parent education middle school',
    'PAR_ED_PCT_HS': '% parent education high school',
    'PAR_ED_PCT_PS': '% parent education post secondary',
    'DEP_INC_AVG': 'Avg income dependent stu',
    'IND_INC_AVG': 'Avg income independent stu',
    'DEBT_MDN': 'Median debt',
    'GRAD_DEBT_MDN': 'Median debt complete',
    'WDRAW_DEBT_MDN': 'Median debt non-completer',
}
# print len(col_desc)


46


In [101]:
data = data[sorted(col_desc.keys())]

In [102]:
# add column that indicates whether it's a less than 4yr college
data['L4_COLLEGE'] = data.C150_4_POOLED.isnull()

# combine completion data for 4 year and <4 year institution
data['C150'] = pd.concat([data.C150_4_POOLED.dropna(), data.C150_L4_POOLED.dropna()]).reindex_like(data)

# combine retention data for 4 year and <4 year institution
data['RET_FT'] = pd.concat([data.RET_FT4.dropna(), data.RET_FTL4.dropna()]).reindex_like(data)

# combine net price title iv for public and private
data['NPT4'] = pd.concat([data.NPT4_PRIV.dropna(), data.NPT4_PUB.dropna()]).reindex_like(data)
data['NUM4'] = pd.concat([data.NUM4_PRIV.dropna(), data.NUM4_PUB.dropna()]).reindex_like(data)


In [103]:
# clean up extra columns after combining
del_columns = ['NPT4_PUB', 'NPT4_PRIV', 'NUM4_PUB', 'NUM4_PRIV', 'C150_4_POOLED', 'C150_L4_POOLED']
for col in del_columns:
    if col in data.keys():
        del data[col]
        del col_desc[col]

col_desc['L4_COLLEGE'] = '<4 years college'
col_desc['C150'] = 'Completion'
col_desc['RET_FT'] = 'Retention'
col_desc['NPT4'] = 'Avg net price Title IV'
col_desc['NUM4'] = 'Num Title IV student'

data = data[~data['C150'].isnull()]
data = data[~data['RET_FT'].isnull()]

# remove data containing 'PrivacySuppressed'
for col in col_desc.keys():
    if data.dtypes[col] == 'object':
        data[col] = data[col].replace(['PrivacySuppressed'], [float('NaN')]).astype(float)


print "Num data after removing missing completion and retention rate: {}".format(len(data))
print "Num features: {}".format(len(data.columns))
print data.shape


Num data after removing missing completion and retention rate: 6007
Num features: 45
(6007, 45)


In [104]:
subset_data = data[['CONTROL', 'DEBT_MDN', 'DEP_INC_AVG', 'GRAD_DEBT_MDN', 'IND_INC_AVG', 'INEXPFTE', 'PAR_ED_PCT_1STGEN', 
              'PAR_ED_PCT_HS', 'PAR_ED_PCT_MS', 'PAR_ED_PCT_PS', 'PCTFLOAN', 'PCTPELL', 'UG25abv', 'UGDS', 
              'WDRAW_DEBT_MDN', 'L4_COLLEGE', 'NPT4', 'NUM4', 'C150', 'RET_FT']].dropna()
Y_all = subset_data[['C150', 'RET_FT']]
y1 = np.array(subset_data['C150'])
y2 = np.array(subset_data['RET_FT'])
X_all = subset_data[['CONTROL', 'DEBT_MDN', 'DEP_INC_AVG', 'GRAD_DEBT_MDN', 'IND_INC_AVG', 'INEXPFTE', 'PAR_ED_PCT_1STGEN', 
              'PAR_ED_PCT_HS', 'PAR_ED_PCT_MS', 'PAR_ED_PCT_PS', 'PCTFLOAN', 'PCTPELL', 'UG25abv', 'UGDS', 
              'WDRAW_DEBT_MDN', 'L4_COLLEGE', 'NPT4', 'NUM4']]
print X_all.shape
print subset_data['DEBT_MDN'].describe()
print subset_data['UGDS'].describe()
print subset_data['NUM4'].describe()
print subset_data['UG25abv'].describe()

(4625, 18)
count     4625.000000
mean     11961.656649
std       5586.002797
min       1280.000000
25%       7751.000000
50%       9833.000000
75%      16018.000000
max      31000.000000
Name: DEBT_MDN, dtype: float64
count      4625.000000
mean       3143.174270
std        6260.687469
min           3.000000
25%         223.000000
50%         862.000000
75%        3107.000000
max      166816.000000
Name: UGDS, dtype: float64
count     4625.000000
mean       327.307459
std        525.163030
min          1.000000
25%         50.000000
50%        145.000000
75%        385.000000
max      13183.000000
Name: NUM4, dtype: float64
count    4625.000000
mean        0.381306
std         0.213408
min         0.000600
25%         0.223800
50%         0.371300
75%         0.522400
max         0.978300
Name: UG25abv, dtype: float64


In [105]:
from sklearn.decomposition import PCA
# features related to cost in $ (debt, cost, income)
cost_cols = ['DEBT_MDN', 'DEP_INC_AVG', 'GRAD_DEBT_MDN', 'IND_INC_AVG', 'INEXPFTE', 'WDRAW_DEBT_MDN', 'NPT4']
pca = PCA(n_components=len(cost_cols))
pca.fit(X_all[cost_cols])

# print pca.components_
# Print the amount of variance in the data contained in each dimension
print pca.explained_variance_ratio_

# features related to pct of student/family composition (pct parent education, loan, pct age >25)
pct_cols = ['PAR_ED_PCT_1STGEN', 'PAR_ED_PCT_HS', 'PAR_ED_PCT_MS', 'PAR_ED_PCT_PS', 'PCTFLOAN', 'PCTPELL', 'UG25abv']
pca = PCA(n_components=len(pct_cols))
pca.fit(X_all[pct_cols])
print pca.explained_variance_ratio_

[ 0.75208444  0.10635322  0.06195659  0.04306176  0.02652898  0.00695459
  0.00306043]
[  5.36880641e-01   2.99177189e-01   1.02480805e-01   5.14169044e-02
   1.00444609e-02   5.01590600e-19   2.14970435e-19]


In [106]:
pca = PCA(n_components=4)
pca.fit(X_all[cost_cols])
reduced_data_cost = pca.transform(X_all[cost_cols])

pca = PCA(n_components=4)
pca.fit(X_all[pct_cols])
reduced_data_pct  = pca.transform(X_all[pct_cols])

print reduced_data_cost.shape
print reduced_data_pct.shape

(4625, 4)
(4625, 4)


In [107]:
# print reduced_data_cost[:5]
# print reduced_data_pct[:5]

# print X_all[['CONTROL', 'L4_COLLEGE', 'UGDS', 'NUM4']]

pd_cost = pd.DataFrame(reduced_data_cost, columns=['cost1', 'cost2', 'cost3', 'cost4'])
pd_pct  = pd.DataFrame(reduced_data_pct,  columns=['pct1', 'pct2', 'pct3', 'pct4'])

X = pd.DataFrame(index=range(0, len(X_all)))

X = X.join(pd_cost)
X = X.join(pd_pct)

X = X.join(pd.DataFrame(data=np.array(pd.get_dummies(X_all['CONTROL'], prefix='type')), index=range(0, len(X_all))))
X = X.join(pd.DataFrame(data=np.array(X_all['L4_COLLEGE'].astype(int)), columns=['L4_COLLEGE']))
X = X.join(pd.DataFrame(data=np.array(X_all[['UGDS', 'NUM4']]), index=range(0, len(X_all)), columns=['UGDS', 'NUM4']))
print X.shape
print X.columns
        
# X = np.concatenate((reduced_data_cost, reduced_data_pct), axis=1)[:5]

(4625, 14)
Index([     u'cost1',      u'cost2',      u'cost3',      u'cost4',
             u'pct1',       u'pct2',       u'pct3',       u'pct4',
                   0,             1,             2, u'L4_COLLEGE',
             u'UGDS',       u'NUM4'],
      dtype='object')


In [108]:
from sklearn import metrics
from sklearn import cross_validation as cv
from sklearn.tree import DecisionTreeRegressor

X1_train, X1_test, y1_train, y1_test = cv.train_test_split(X, y1, train_size=0.8)
X2_train, X2_test, y2_train, y2_test = cv.train_test_split(X, y2, train_size=0.8)


# print X1_train.shape
# print X1_test.shape
# print y1_train.shape
# print y1_test.shape
# print y2_train.shape
# print y2_test.shape

reg1 = DecisionTreeRegressor(max_depth=8)
reg2 = DecisionTreeRegressor(max_depth=8)

reg1.fit(X1_train, y1_train)
reg2.fit(X2_train, y2_train)

DecisionTreeRegressor(criterion='mse', max_depth=8, max_features=None,
           max_leaf_nodes=None, min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, presort=False, random_state=None,
           splitter='best')

In [109]:
print reg1.predict(np.array(X[:4]))
print y1[:4]
print reg2.predict(np.array(X[:4]))
print y2[:4]

def print_r2score(reg, X, y, test=False):
    t = 'test' if test else 'train'
    print "R2 score on {} data: {}".format(t, metrics.r2_score(y, reg.predict(np.array(X))))
    
print "Completion"
print "R2 score on train data: {}".format(metrics.r2_score(y1_train, reg1.predict(np.array(X1_train))))
print "R2 score on test data:  {}".format(metrics.r2_score(y1_test,  reg1.predict(np.array(X1_test))))
print "Retention"
print "R2 score on train data: {}".format(metrics.r2_score(y2_train, reg2.predict(np.array(X2_train))))
print "R2 score on test data:  {}".format(metrics.r2_score(y2_test,  reg2.predict(np.array(X2_test))))


[ 0.33728794  0.42193621  0.25615599  0.42193621]
[ 0.30871831  0.50854981  0.2857381   0.47821135]
[ 0.56958471  0.70837059  0.42401923  0.70837059]
[ 0.6314  0.8016  0.375   0.8098]
Completion
R2 score on train data: 0.756453922279
R2 score on test data:  0.618693401856
Retention
R2 score on train data: 0.5347523883
R2 score on test data:  0.180102471822


In [110]:
from sklearn.grid_search import GridSearchCV

parameters = {'max_depth': range(1,10) } # , 'min_samples_leaf': [4,5,6,7]}
scorer = metrics.make_scorer(metrics.r2_score, greater_is_better=True)
reg = DecisionTreeRegressor()
best_reg = GridSearchCV(reg, parameters, scoring=scorer, cv=2)
best_reg.fit(X1_train, y1_train)
print best_reg.best_params_

print_r2score(best_reg, X1_train, y1_train)
print_r2score(best_reg, X1_test, y1_test, test=True)

reg = DecisionTreeRegressor()
best_reg = GridSearchCV(reg, parameters, scoring=scorer, cv=4)
best_reg.fit(X2_train, y2_train)
print best_reg.best_params_

print_r2score(best_reg, X2_train, y2_train)
print_r2score(best_reg, X2_test, y2_test, test=True)


# reg = DecisionTreeRegressor(max_depth=16)
# reg.fit(X_train, y1_train)
# print_r2score(reg, X_train, y1_train)
# print_r2score(reg, X_test, y1_test, test=True)

{'max_depth': 7}
R2 score on train data: 0.718886981382
R2 score on test data: 0.649123207112
{'max_depth': 4}
R2 score on train data: 0.314488331437
R2 score on test data: 0.304410500572


In [111]:
from sklearn.svm import SVR

params = {'C': np.logspace(-1, 1, 2), 'gamma': np.logspace(-1, 1, 2)} # , 'gamma': np.logspace(-2, 2, 5)
reg = SVR()
best_reg = GridSearchCV(reg, params, scoring=scorer, cv=3)
best_reg.fit(X1_train, y1_train)

print best_reg.best_params_
print_r2score(best_reg, X1_train, y1_train)
print_r2score(best_reg, X1_test, y1_test, test=True)

{'C': 10.0, 'gamma': 0.10000000000000001}
R2 score on train data: 0.838703231253
R2 score on test data: -0.00200098005302


In [112]:
reg = SVR()
params = {'C': np.logspace(-1, 1, 2), 'gamma': np.logspace(-1, 1, 2), 'epsilon': np.logspace(-1, 1, 2)}
best_reg = GridSearchCV(reg, params, scoring=scorer, cv=3)
best_reg.fit(X2_train, y2_train)

print best_reg.best_params_
print_r2score(best_reg, X2_train, y2_train)
print_r2score(best_reg, X2_test, y2_test, test=True)

{'epsilon': 0.10000000000000001, 'C': 0.10000000000000001, 'gamma': 0.10000000000000001}
R2 score on train data: 0.481684218571
R2 score on test data: -0.00312067687584


In [96]:
# print X1_train.columns

# for col in X1_train.columns:
#     print col
#     plt.scatter(X1_train[col], y1_train, c='b', s=2, linewidths=0)
#     plt.show()

    

In [97]:
# for col, data in subset_data.iteritems():
#     print col

# cols = ['DEBT_MDN', 'DEP_INC_AVG', 'GRAD_DEBT_MDN', 'IND_INC_AVG', 'INEXPFTE']
# for col in cols:
#     print col
#     plt.scatter(np.array(subset_data[col]), subset_data['C150'], c='b', s=2, linewidths=0)
#     plt.show()
    


In [113]:
# print subset_data[subset_data['CONTROL'] == 1].shape
# print subset_data[subset_data['CONTROL'] == 2].shape
# print subset_data[subset_data['CONTROL'] == 3].shape

c1_data = subset_data

X = pd.DataFrame(index=c1_data.index)

X = X.join(pd.DataFrame(data=pd.get_dummies(c1_data['CONTROL'], prefix='type')))
X = X.join(pd.DataFrame(data=X_all['L4_COLLEGE'].astype(int)))
# X = X.join(pd.DataFrame(data=np.array(X_all[['UGDS', 'NUM4']]), index=range(0, len(X_all)), columns=['UGDS', 'NUM4']))

cost_cols = ['DEBT_MDN', 'DEP_INC_AVG', 'GRAD_DEBT_MDN', 'IND_INC_AVG', 'INEXPFTE', 'WDRAW_DEBT_MDN', 'NPT4']
X = X.join(pd.DataFrame(data=c1_data[cost_cols]))
pct_cols = ['PAR_ED_PCT_1STGEN', 'PAR_ED_PCT_HS', 'PAR_ED_PCT_MS', 'PAR_ED_PCT_PS', 'PCTFLOAN', 'PCTPELL', 'UG25abv']
X = X.join(pd.DataFrame(data=c1_data[pct_cols]))
y1 = c1_data['C150']
y2 = c1_data['RET_FT']

# print y1
# print X.columns
# print X.index
# print X

X1_train, X1_test, y1_train, y1_test = cv.train_test_split(X, y1, train_size=0.8)
X2_train, X2_test, y2_train, y2_test = cv.train_test_split(X, y2, train_size=0.8)

parameters = {'max_depth': range(3,15) } # , 'min_samples_leaf': [4,5,6,7]}
scorer = metrics.make_scorer(metrics.r2_score, greater_is_better=True)
reg = DecisionTreeRegressor()
best_reg = GridSearchCV(reg, parameters, scoring=scorer, cv=3)
best_reg.fit(X1_train, y1_train)
print best_reg.best_params_

print_r2score(best_reg, X1_train, y1_train)
print_r2score(best_reg, X1_test, y1_test, test=True)


reg = DecisionTreeRegressor()
best_reg = GridSearchCV(reg, parameters, scoring=scorer, cv=3)
best_reg.fit(X2_train, y2_train)
print best_reg.best_params_

print_r2score(best_reg, X2_train, y2_train)
print_r2score(best_reg, X2_test, y2_test, test=True)


{'max_depth': 6}
R2 score on train data: 0.673756580985
R2 score on test data: 0.582673821795
{'max_depth': 5}
R2 score on train data: 0.360852885325
R2 score on test data: 0.259410287908


In [115]:
from sklearn.neighbors import KNeighborsRegressor

parameters = {'n_neighbors': [5,6,7,8,9,10]}
reg = KNeighborsRegressor()
best_reg = GridSearchCV(reg, parameters, scoring=scorer, cv=3)
best_reg.fit(X1_train, y1_train)
print best_reg.best_params_
print_r2score(best_reg, X1_train, y1_train)
print_r2score(best_reg, X1_test, y1_test, test=True)

reg = KNeighborsRegressor()
best_reg = GridSearchCV(reg, parameters, scoring=scorer, cv=3)
best_reg.fit(X2_train, y2_train)
print best_reg.best_params_
print_r2score(best_reg, X2_train, y2_train)
print_r2score(best_reg, X2_test, y2_test, test=True)



{'n_neighbors': 6}
R2 score on train data: 0.703867780888
R2 score on test data: 0.591763570403
{'n_neighbors': 10}
R2 score on train data: 0.420926436251
R2 score on test data: 0.314977896738
